# Create A Sample Manifest File for STAR-Fusion NF Workflow

### Jenny Smith 
### Sept 20, 2020

In [44]:
import os
import pathlib
import socket
import numpy as np
import pandas as pd
import re
import boto3
import botocore
import create_sample_sheet


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# https://realpython.com/python-pathlib/
host = socket.gethostname()
if re.search("MacBook|mbp.+fhcrc.org", host):
    TARGET="/Users/work/fast_drive/workingDir/TARGET/AML_TARGET/"

elif re.search("gizmo|rhino", host):
    TARGET="/fh/fast/meshinchi_s/workingDir/TARGET/AML_TARGET/"

TARGET=pathlib.Path(TARGET)
TARGET

PosixPath('/Users/work/fast_drive/workingDir/TARGET/AML_TARGET')

In [3]:
pathlib.Path().cwd()

PosixPath('/Users/work/STAR-fusion-NF/python_code')

# 1. Read in the Manifest file

In [4]:
# [{print(f)}for file in TARGET.joinpath("SequencingDataMatrix/").iterdir()]

manifest_file = TARGET.joinpath("SequencingDataMatrix/TARGET_AML_Ribodepleted_Master_Manifest_8.5.20.csv")


manifest = pd.read_csv(manifest_file, 
                       engine='python')

manifest.head()
manifest.shape


,Unnamed: 0,Sample,Final_Patient_ID,PATIENT_ID_Original,USI,Reg.,Protocol,AML_Subtype,Group,Batch,Library,Time_point,Tissue,Protocol2,Primary.Fusion.CNV,Additional.Fusions.CNV
0,1,Kasumi.AZA.D11.03A.01R,Kasumi-AZA-D11-03A-01R,Kasumi-AZA-D11-03A-01R,Kasumi,NaN,CellLine,CellLine,CellLine,dx1,A75511,CellLine,CellLine,NaN,NaN,NaN
1,2,Kasumi.AZA.D5.03A.01R,Kasumi-AZA-D5-03A-01R,Kasumi-AZA-D5-03A-01R,Kasumi,NaN,CellLine,CellLine,CellLine,dx1,A75509,CellLine,CellLine,NaN,NaN,NaN
2,3,Kasumi.D1.03A.01R,Kasumi-D1-03A-01R,Kasumi-D1-03A-01R,Kasumi,NaN,CellLine,CellLine,CellLine,dx1,A75507,CellLine,CellLine,NaN,NaN,NaN
3,4,MV4.11.AZA.D11.03A.01R,MV4-11-AZA-D11-03A-01R,MV4-11-AZA-D11-03A-01R,MV4,NaN,CellLine,CellLine,CellLine,dx1,A75512,CellLine,CellLine,NaN,NaN,NaN
4,5,MV4.11.AZA.D5.03A.01R,MV4-11-AZA-D5-03A-01R,MV4-11-AZA-D5-03A-01R,MV4,NaN,CellLine,CellLine,CellLine,dx1,A75510,CellLine,CellLine,NaN,NaN,NaN


(2346, 16)

In [86]:
type(samples_cbf) is str

True

In [5]:
#table for the different Batchs 
manifest.Batch.value_counts()

dx1      1117
dx2       457
rlps2     302
ds1       228
rlps1     198
rlps3      36
rlps4       7
Name: Batch, dtype: int64

In [6]:
polyA_RBD_file = TARGET.joinpath("SequencingDataMatrix/TARGET_AML_RBD_PolyA_RNAseq_AWS_S3_Fastq_Manifest_8.5.20.csv")
polyA_RBD_manifest = pd.read_csv(polyA_RBD_file)


# polyA_RBD_manifest.head()
polyA_RBD_manifest.shape

(2777, 16)

# 2. Subset for Required Samples

In [7]:
manifest.fillna(value="",inplace=True) #Stella is an NA

In [8]:
ds_aml = manifest[manifest.Batch.str.contains("ds")]

# ds_aml.head()
ds_aml.shape #228 rows

(228, 16)

In [9]:
# & cbf_aml.Time_point.str.contains("diagnostic")
cbf_aml = polyA_RBD_manifest[polyA_RBD_manifest.AML_Subtype.str.contains("CBFB-MYH11|RUNX1-RUNX1T1") & polyA_RBD_manifest.Time_point.str.contains("diagnostic")]

# cbf_aml.head()
# cbf_aml.shape
cbf_aml.AML_Subtype.value_counts()
# cbf_aml.to_csv("CBFB_MYH11_RUNX1_RUNX1T1_Fastqs_Manifest.csv")

RUNX1-RUNX1T1    215
CBFB-MYH11       185
Name: AML_Subtype, dtype: int64

In [10]:
cbf_aml.Time_point.value_counts()
cbf_aml.Group.value_counts()
cbf_aml.Lib_Prep.value_counts()

diagnostic    400
Name: Time_point, dtype: int64

AML    400
Name: Group, dtype: int64

RBS      283
PolyA    117
Name: Lib_Prep, dtype: int64

# 3. Run the Sample Sheet Script

In [11]:
bucket="fh-pi-meshinchi-s"
prefix="SR/picard_fq2/"
# ?create_sample_sheet.create_sample_sheet

In [ ]:
samples = " ".join(ds_aml.Sample.tolist())
# type(samples)

create_sample_sheet.create_sample_sheet(bucket_name=bucket, prefix_name=prefix, samples=samples, filename="test_sheet.txt")

# 4. Create Presigned URLs

In [12]:
# creds_path = pathlib.Path("~/.aws/credentials").expanduser()
# dir(creds_path)
# creds_path

# # [print(f) for f in creds_path.iterdir()]
# os.environ['AWS_SHARED_CREDENTIALS_FILE'] = str(creds_path)

In [12]:
accountinfo = boto3.client('sts').get_caller_identity()
print(f"Account is: {accountinfo}")

Account is: {'UserId': 'AIDA3AANZ4DLNKFSCPE5D', 'Account': '755943203030', 'Arn': 'arn:aws:iam::755943203030:user/terraform/jlsmith3', 'ResponseMetadata': {'RequestId': '286c8115-6261-46a5-b496-700178bfb323', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '286c8115-6261-46a5-b496-700178bfb323', 'content-type': 'text/xml', 'content-length': '415', 'date': 'Fri, 25 Sep 2020 23:11:06 GMT'}, 'RetryAttempts': 0}}


In [28]:
# type(os.environ)
# for k in os.environ.keys():
#     print(k)

# type(os.getenv("AWS_SHARED_CREDENTIALS_FILE"))
# type(os.getenv("AWS_CONFIG_FILE"))

if not os.getenv("AWS_SHARED_CREDENTIALS_FILE") is None:
    os.getenv("AWS_SHARED_CREDENTIALS_FILE")
if not os.getenv("AWS_CONFIG_FILE") is None:
    os.getenv("AWS_CONFIG_FILE")

'~/.aws/credentials'

'~/.aws/config'

In [16]:
samples_cbf = " ".join(cbf_aml.fastq_sample_filename.tolist())
forURLs = create_sample_sheet.create_sample_sheet(bucket_name=bucket, prefix_name=prefix, samples=samples_cbf, write=False)

There are 400 Fastq files.


In [21]:

#Not working in Jupyter lab. works on command line and ipython command line. exact same code. 
outfile="CBFB_MYH11_RUNX1_RUNX1T1_presigned_URLs.txt"
responses = []
for R1, R2 in zip(forURLs.R1,forURLs.R2):
    end = len(R1)
    start = len(bucket) + 6
    expiration = 604800
    object_1 = R1[start:end]
    object_2 = R2[start:end]
    response1 = create_sample_sheet.create_presigned_url(bucket_name=bucket,
                                             object_name=object_1,
                                             expiration=expiration)
    response2 = create_sample_sheet.create_presigned_url(bucket_name=bucket,
                                             object_name=object_2,
                                             expiration=expiration)
    responses.append(response1)
    responses.append(response2)

len(responses)
# with open(outfile, "w") as outfile:
#     outfile.write("\n".join(responses))

20